In [1]:
import os
import torch

DIR= r'/Token_Classification/'
path_file= os.path.join(DIR, 'model_classifier.pth')

from transformers import AutoTokenizer
bert_model = "distilbert-base-uncased"

from classes import Model_TC 

In [2]:
def inference(input, path_file, bert_model):
    labels_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

    tokenizer= AutoTokenizer.from_pretrained(bert_model)
    input_ts = tokenizer(input, is_split_into_words=False, return_tensors='pt')
    
    model = Model_TC.TokenClassifier(bert_model, num_labels=len(labels_list))
  
    checkpoint = torch.load(path_file)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    model.eval()
    output = model(**input_ts)[0].detach()
    
    prediction = output.argmax(dim=-1).tolist()
    
    for t, p in zip(input_ts['input_ids'][0].tolist(), prediction):
        if p > 0:
            w = tokenizer.decode(t)
            print(w, labels_list[p])

In [3]:
input1 = input('Enter a sentence:\n ')
inference(input1, path_file, bert_model)
input2 = input('Enter a sentence:\n ')
inference(input2, path_file, bert_model)
print('done')

Enter a sentence:
 The italian Giorgio Parisi won the Nobel Prize in 2021.
italian B-MISC
giorgio B-PER
paris I-PER
##i I-PER
nobel B-MISC
Enter a sentence:
 Martina lives in Rome and she is working at Google.
martina B-PER
rome B-LOC
google B-ORG
done
